<a href="https://colab.research.google.com/github/image0of0pie/machine-learning-projects/blob/master/Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting drive and importing libraries and model.....

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timeit
import tensorflow as tf
import seaborn as sns
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [ ]:
base_path="/content/drive/My Drive/Machine Learning Projects/Spatiotemporal forecasting using BLSTM+spatial interpolation/Data/"

# Defining preprocessing fnc.

# Importing files

In [ ]:
test=pd.read_excel(base_path+'Alipur.xlsx')
test=test.iloc[1279:17495]
test=test.reset_index()

In [ ]:
COLUMNS=['From Date','To Date','PM2.5','PM10','NO','NO2']
TRAIN_COLUMNS=['PM10','PM2.5','NO','NO2']
final_data=pd.DataFrame()
for col in COLUMNS:
    final_data[col]=new_test[col]
train_data=pd.DataFrame()
for col in TRAIN_COLUMNS:
    train_data[col]=new_test[col]

In [ ]:
colormap = plt.cm.viridis
plt.figure(figsize=(4,4))
plt.title('Correlation of Pollutants', y=1.05, size=15)
sns.heatmap(train_data.astype(float).corr(),vmax=1.0, square=True, cmap=colormap, annot=True)

# Test

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix>len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [ ]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [ ]:
dataset = hstack((in_seq1, in_seq2, out_seq))

In [ ]:
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

In [ ]:
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

In [ ]:
history=model.fit(X,y,epochs=300)